In [143]:
#This block is for all the libraries that the program require.
import math
import en_core_web_sm
import numpy as np
import pandas as pa
import os 
import nltk
from bs4 import BeautifulSoup
import re, string, unicodedata
import nltk
#nltk.download('averaged_perceptron_tagger')
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import inflect
import textstat
#nltk.download('wordnet')
from gensim.test.utils import common_texts,get_tmpfile
from gensim.models import Word2Vec
import gensim
from nltk.tag import RegexpTagger
from collections import OrderedDict 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy
from collections import Counter
from nltk.stem import PorterStemmer

import math
import numpy as np
import os 
import nltk
from bs4 import BeautifulSoup
#Preprocessing packages
import re, string, unicodedata
import nltk
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from gensim.test.utils import common_texts,get_tmpfile
from gensim.models import Word2Vec
import gensim
#Import necessary Pakages
from os import listdir
import os
import itertools
import sklearn
import inflect
import random

In [72]:
#This block contains from reading teh master file to loading content from teh directory into teh dataframe.

def load_nd_save(master_file_path, books_file_path):
    meta_data = pa.read_csv(master_file_path,sep = ";", engine = 'python', 
                            header = 0, encoding = 'ISO-8859-1')

    #meta_data['book_id'].isna().sum() # to check if there are any  na values in book_id column

    # removing the epub extension
    meta_data['book_id'] = meta_data['book_id'].replace(regex=['.epub'],value= '-content.html')

    # Loading the content into data frames
    files_dir = books_file_path
    for i in meta_data.index:
        try:
            f = str(meta_data.loc[i,'book_id'])
            soup = BeautifulSoup(open(files_dir + f,encoding="ISO-8859-1"),'html.parser')
            meta_data.loc[i,'content'] = soup.text
        except:
            continue
    meta_data = meta_data.dropna(axis = 0)
    meta_data = meta_data.replace(np.nan, 0)
    print(meta_data)
    return (meta_data)

In [73]:
def preprocess(text):
    text = re.sub('\[[^]]*\]', '', text) #remove_between_square_brackets(text)
    words = nltk.word_tokenize(text)
    new_words = [] 
    #new_words = list([new_words.append(unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')) for word in words])
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    words = [c.lower() for c in new_words]
    getVals = list([val for val in words if val.isalpha()])
    words = getVals
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    #new_words = list([new_words.append(word) for word in words if word not in stopwords.words('english')])
    #lemmatizer = WordNetLemmatizer()
    #lemmas = []
    #for word in new_words:
        #lemma = lemmatizer.lemmatize(word)
        #lemmas.append(lemma)
    return new_words

In [74]:
 
# This block will pre-process for each book and save it into a dictionary with 
#all the books and it's tokens and in a dataframe as a one string as well
def call_preprocess(no_of_books, meta_data):
    # pre-processing first 50 books
    a = meta_data.loc[0:no_of_books,]
    dit = {}
    for i in a.index:
        words = preprocess(a.loc[i,'content'])
        dit[a.loc[i,'book_id']] = list(words)
        a.loc[i,'preprocessed_content'] = ' '.join(map(str,words))
    return(a,dit)

In [148]:
def features(a, no_of_books):
    nlp = en_core_web_sm.load()
    # Extracting few features
    # calculating total no of words in each book
    analyser = SentimentIntensityAnalyzer()
    a['content_word_count'] = a['content'].apply(lambda x: len(x.strip().split()))

    # calculating unique no of words in each book
    a['content_unique_words']= a['content'].apply(lambda x:len(set(str(x).split())))

    # calculating the readbility  score here
    a['flesch_score'] = a['content'].apply(lambda x : textstat.flesch_reading_ease(x))
    
    a['pre-processed_unique_words']= a['preprocessed_content'].apply(lambda x:len(set(str(x).split(" "))))
    #writing_style2 = pa.DataFrame(columns = ['noun','comma','Cardinal_nos','hyphen','Plural_nouns','Past_Tense_Verbs','Adverbs','Exclamation','Articles','Male_Pronoun','Personal_Pronoun','Gerund','Semicolon','Colon','Adjectives','Interjections','Possessive_Pronoun','Female_Pronoun'])
    
    # calculating teh no of sentences in each book
    a['no_sentences'] = a['content'].apply(lambda x: len(nltk.sent_tokenize(x)))
    
    li = []
    cordConj_l = []
    subConj_l = []
    senti_l = pa.DataFrame()
    period_l = []
    char_l = []
    for i in a.index:
        regexp_tagger = RegexpTagger([ (r'(aha|ahem|ahh|ahoy|alas|arg|aw|bam|bingo|blah|boo|bravo|brrr|cheers|congratulations|dang|drat|darn|duh|eek|eh|encore|eureka|fiddlesticks|gadzooks|gee|gee|whiz|golly|goodbye|good|grief|gosh|Shh|Please|please|Psst|Shoo|Hey|Oh|Yo|Here|Ahem|Encore|Hush|Scat|No|Silence|Enough|Yuck|Eww|Aww|Ouch|Oh|Ah|Ugh|Phew|Phooey|Rats|Yippee|Aw|Blah|Brr|Eek|Good|Grief|Alas|Bingo|Bravo|Eureka|Crikey|Gee|Golly|Gosh|Hmm|Holy|cow|Aha|Oh|Huh|Duh|Ahh|Well)$','a') , # Interjections
            (r'^-?[0-9]+(.[0-9]+)?$', 'b'), #  for cardinal numbers
            (r'(The|the|A|a|An|an)$', 'c'),  # for articles
             (r'.*able$', 'd'),                # adjectives
             (r'.*ly$', 'e'),                  # adverbs
             (r'.*s$', 'f'),                   # plural nouns
              (r'.*ing$', 'g'),                # gerunds
              (r'.*ed$', 'h'),                 # past tense verbs
              (r'(She|she|her|Her)$', 'i') ,  # female pronouns
             (r'(he|HIM|him|his|HIS|Him|His|He)$', 'j'), # Male pronouns
             (r'(I|i|you|You|It|it|We|we|They|they)$', 'k'), # Personal Pronoun
             (r'(Mine|Mine|yours|Yours|Hers|hers|Theirs|theirs|their|Their|Its|its|Ours|ours|Ones)$', 'l'), # Possessive Pronoun   
                (r'-', 'm'),
                (r';','n'),
                (r':','o'),
                (r'!', 'p'),
                (r',','q'),
                (r'(.*)','r')  # nouns, basically whatever is left 
            ])
        x = nltk.word_tokenize(a.loc[i,'content'])
        ######### Mr. chandan's part
        #words_Sent = nltk.word_tokenize(merged_df['content'].values[i])
        words_Sent = np.unique(x)
        num = 0.5*len(np.unique(words_Sent))
        words_Sent = random.sample(list(words_Sent), int(num))
        temp = ' '.join(map(str,words_Sent))
        #for j in words_Sent: 
         #   temp = temp+(j)+" "
        sent = analyser.polarity_scores(temp)
        senti_l = senti_l.append(sent, ignore_index= True) 
        #merged_df['sent_scores'].values[i] = sent
        ########### now Priyanka's part
        y = nltk.pos_tag(x)
        CCcnt=0
        subCnt=0
        cordConj = [item for item in y if 'CC' in item]
        subConj = [item for item in y if 'IN' in item]
        cordConj_l.append(len(cordConj))
        subConj_l.append(len(subConj))
        tk = regexp_tagger.tag(x)
        tag_fd = nltk.FreqDist(tag for (word, tag) in tk)
        sorted_items = OrderedDict(sorted(tag_fd.items())) 
        li.append(sorted_items.values())
        period_l.append(a.loc[i,'content'].count('.'))
        ############# Mr Chandan part 2
        def process_words(words):
            ps = PorterStemmer()
            # convert to lower case
            words = [w.lower() for w in words]
            words = [word for word in words if word.isalpha()]
            stems = [ps.stem(w) for w in words]
            return np.unique(stems)

        #for i in range(len(merged_df)):
        doc = nlp(a.loc[i,'content'])
        # Text and label of named entity span
        taggings = pa.DataFrame([(ent.text, ent.label_) for ent in doc.ents])
        taggings.columns = ['name','tag']
        tags = taggings[taggings.tag=="PERSON"]
        counts = dict(Counter(tags.name))
        chars = []
        for val in counts:
            if(counts[val]>1):
                chars.append(val)
        char_l.append(len(process_words(np.unique(chars))))
        
        
        ########################
        
    writing_style = pa.DataFrame(li)
    #writing_style.columns = sorted_items.keys()
    #changing the column names of writing style dataframe
    writing_style.columns = ['Interjections','Cardinal_Nums','Articles','Adjectives','Adverbs','Plural_Nouns','Gerunds','Past_tense_verbs','Female_Pronoun','Male_Pronoun','Personal_Pronoun','Possessive_Pronoun','Hyphen','semi-colon','colon','Exclamation','Comma','Noun']
    writing_style['coordinating_conjunctions'] = cordConj_l
    writing_style['subcord_conjunctions'] = subConj_l
    writing_style['char_count'] = char_l
    #writing_style['sent_scores'] = senti_l
    writing_style['period'] = period_l
    final_df = pa.concat([writing_style,senti_l], axis = 1)
    return a,final_df


In [77]:
#calculates word vectors
def wordvectors(dit):    
    # calculating word vectors for all the books
    my_df = pa.DataFrame()
    for key in dit:
        d = [dit[key]]
        word2veca = gensim.models.Word2Vec([dit[key]] ,min_count = 1, size = 15, window = 8, sg = 1 , seed = 123, negative = 0, iter = 100, workers = 10)
        words = pa.DataFrame(word2veca.wv.vectors, index=word2veca.wv.index2word)
        words = words.values.sum(axis = 0) / words.shape[0]
        my_df.loc[:,str(key)] = words

    f = my_df.T
    f.reset_index(level=0, inplace=True)
    return f

word_vectors = pa.DataFrame(word2veca.wv.vectors, index=word2veca.wv.index2word)
word_vectors

In [78]:
def features_append(writing_style,f, a, no_of_books):
    # appending all the other fetaures to teh word2vec
    f['label'] =  a.loc[0:no_of_books,'guten_genre']
    f['flesch_score'] = a.loc[0:no_of_books,'flesch_score']
    f['token_ratio'] = a.loc[0:no_of_books,'content_unique_words']/a.loc[0:no_of_books,'content_word_count']
    f['no_of_sentences'] = a.loc[0:no_of_books,'no_sentences']
    gum = pa.concat([f,writing_style],axis = 1)
    gum = gum.replace(np.nan, 0)
    print(gum)
    return gum

In [152]:
def train_n_predict(gum):
    cols2exclude = [0,16,21,39] # These are teh features that we have excluded and not decided to train and base our model on.
    y = gum.loc[:,'label']
    X = preprocessing.scale(gum.iloc[:, ~gum.columns.isin(gum.columns[cols2exclude])])
    X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.3, random_state = 0)

    gnb = GaussianNB()
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    train_pred = gnb.fit(X_train,y_train).predict(X_train)
    print("********* Following results on test data ***********")
    print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))
    bn = y_test.tolist()
    for i in range(len(y_pred)):
          print("actual genre is ",bn[i],"but predicted is ",y_pred[i])


    print("f1 score is ", f1_score(y_test, y_pred, average='weighted'))

    print( "********** Following data on Training data *********")
    print("Number of mislabeled points out of a total %d points : %d" % (X_train.shape[0], (y_train != train_pred).sum()))
    for i in range(len(train_pred)):
          print("actual genre is ",y_train.tolist()[i],"but predicted is ",train_pred[i])
    print("f1 score is ", f1_score(y_train, train_pred, average='weighted'))


        

In [149]:
%%time
master_file_path =  "D:/DKE-sem/sem2/AML/project/master996n.csv"

books_file_path = "D:/DKE-sem/sem2/AML/project/Gutenberg_English_Fiction_1k/Gutenberg_19th_century_English_Fiction_full/"
no_of_books = 2
meta_data = load_nd_save(master_file_path, books_file_path)
print("successfully loaded")
a, dit = call_preprocess(no_of_books, meta_data)
print("successfully pre-processed")
a, writing_style = features(a, no_of_books)
print("successfully extracted features")
f = wordvectors(dit)

gum = features_append(writing_style,f, a, no_of_books)


X = train_n_predict(gum)
print(X)

                                             Book_Name  \
0    The Mystery of the Boule Cabinet: A Detective ...   
1                                            The Pupil   
2                                       At Love's Cost   
3                               The Heart of the Range   
4                          The Worshipper of the Image   
..                                                 ...   
991                                  David Copperfield   
992                                         Hard Times   
993                          Memoirs of Shelock Holmes   
994                    The Mysterious Affair at Styles   
995                               A Tale of Two Cities   

                                  book_id            guten_genre  \
0                    pg10067-content.html  Detective and Mystery   
1                     pg1032-content.html               Literary   
2                    pg10379-content.html               Literary   
3                    pg10473-co

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

successfully pre-processed


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

successfully extracted features
                  index         0         1         2         3         4  \
0  pg10067-content.html -0.000131 -0.000459  0.000205  0.000471 -0.000475   
1   pg1032-content.html -0.000518 -0.000597  0.000658  0.000299  0.000658   
2  pg10379-content.html -0.000173  0.000168  0.000191  0.000336 -0.000009   

          5         6         7         8  ...   Comma     Noun  \
0 -0.000361 -0.000123 -0.000053  0.000400  ...   53067      0.0   
1  0.000119 -0.000070 -0.000061  0.000394  ...     962  12579.0   
2 -0.000210 -0.000187 -0.000090  0.000099  ...  110445      0.0   

   coordinating_conjunctions  subcord_conjunctions  char_count  period  \
0                       2668                  7753          29    4367   
1                        569                  2266           8     823   
2                       7157                 17707          57    7137   

  compound    neg    neu    pos  
0  -0.9819  0.216  0.577  0.207  
1   0.9994  0.137  0.651 

In [150]:
gum

,index,0,1,2,3,4,5,6,7,8,...,Comma,Noun,coordinating_conjunctions,subcord_conjunctions,char_count,period,compound,neg,neu,pos
0,pg10067-content.html,-0.000131,-0.000459,0.000205,0.000471,-0.000475,-0.000361,-0.000123,-0.000053,0.000400,...,53067,0.0,2668,7753,29,4367,-0.9819,0.216,0.577,0.207
1,pg1032-content.html,-0.000518,-0.000597,0.000658,0.000299,0.000658,0.000119,-0.000070,-0.000061,0.000394,...,962,12579.0,569,2266,8,823,0.9994,0.137,0.651,0.212
2,pg10379-content.html,-0.000173,0.000168,0.000191,0.000336,-0.000009,-0.000210,-0.000187,-0.000090,0.000099,...,110445,0.0,7157,17707,57,7137,-0.9991,0.224,0.575,0.201


In [151]:
gum.columns

Index([                    'index',                           0,
                                 1,                           2,
                                 3,                           4,
                                 5,                           6,
                                 7,                           8,
                                 9,                          10,
                                11,                          12,
                                13,                          14,
                           'label',              'flesch_score',
                     'token_ratio',           'no_of_sentences',
                   'Interjections',             'Cardinal_Nums',
                        'Articles',                'Adjectives',
                         'Adverbs',              'Plural_Nouns',
                         'Gerunds',          'Past_tense_verbs',
                  'Female_Pronoun',              'Male_Pronoun',
                'Personal